In [1]:
import tensorflow as tf
import numpy as np

corpus_raw = 'He is the king . The king is royal . She is the royal queen '

# convert to lower case
corpus_raw = corpus_raw.lower()


In [2]:
words = []
for word in corpus_raw.split():
    if word != '.': # because we don't want to treat . as a word
        words.append(word)
words = set(words) # so that all duplicate words are removed
word2int = {}
int2word = {}
vocab_size = len(words) # gives the total number of unique words
for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

In [4]:
print(word2int['queen'])
print(int2word[0])

0
queen


In [5]:

# raw sentences is a list of sentences.
raw_sentences = corpus_raw.split('.')
sentences = []
for sentence in raw_sentences:
    sentences.append(sentence.split())

print(sentences)

[['he', 'is', 'the', 'king'], ['the', 'king', 'is', 'royal'], ['she', 'is', 'the', 'royal', 'queen']]


In [6]:
WINDOW_SIZE = 2

data = []
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])



In [7]:
# function to convert numbers to one hot vectors
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

x_train = [] # input word
y_train = [] # output word

for data_word in data:
    x_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
    y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))

# convert them to numpy arrays
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)



In [8]:
print(x_train)

[[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]


In [9]:
print(x_train.shape, y_train.shape)

(34, 7) (34, 7)


In [10]:
# making placeholders for x_train and y_train
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))



In [11]:
EMBEDDING_DIM = 5 # you can choose your own number
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) #bias
hidden_representation = tf.add(tf.matmul(x,W1), b1)

W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))




In [12]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) #make sure you do this!

# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))

# define the training step:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)

n_iters = 10000
# train for n_iter iterations

for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_train, y_label: y_train}))



loss is :  4.7203717
loss is :  4.328468
loss is :  4.0159755
loss is :  3.7594314
loss is :  3.5464
loss is :  3.367695
loss is :  3.2154005
loss is :  3.0831301
loss is :  2.9663484
loss is :  2.8621156
loss is :  2.7685668
loss is :  2.684434
loss is :  2.6087244
loss is :  2.5405455
loss is :  2.4790416
loss is :  2.423393
loss is :  2.372842
loss is :  2.3267214
loss is :  2.2844646
loss is :  2.2456071
loss is :  2.209769
loss is :  2.176642
loss is :  2.1459687
loss is :  2.11753
loss is :  2.0911329
loss is :  2.066602
loss is :  2.0437756
loss is :  2.0225034
loss is :  2.0026424
loss is :  1.9840599
loss is :  1.9666331
loss is :  1.9502472
loss is :  1.9347979
loss is :  1.9201909
loss is :  1.9063413
loss is :  1.8931743
loss is :  1.8806219
loss is :  1.8686256
loss is :  1.8571335
loss is :  1.8461003
loss is :  1.8354859
loss is :  1.8252554
loss is :  1.8153781
loss is :  1.8058273
loss is :  1.7965795
loss is :  1.7876136
loss is :  1.7789115
loss is :  1.770457
loss i

loss is :  1.367237
loss is :  1.3670871
loss is :  1.3669376
loss is :  1.3667891
loss is :  1.366641
loss is :  1.3664937
loss is :  1.3663471
loss is :  1.3662009
loss is :  1.3660555
loss is :  1.3659106
loss is :  1.3657665
loss is :  1.365623
loss is :  1.3654801
loss is :  1.3653377
loss is :  1.365196
loss is :  1.365055
loss is :  1.3649147
loss is :  1.3647746
loss is :  1.3646355
loss is :  1.3644968
loss is :  1.3643587
loss is :  1.3642212
loss is :  1.3640844
loss is :  1.3639481
loss is :  1.3638124
loss is :  1.3636773
loss is :  1.3635427
loss is :  1.3634087
loss is :  1.3632753
loss is :  1.3631424
loss is :  1.3630102
loss is :  1.3628784
loss is :  1.3627473
loss is :  1.3626165
loss is :  1.3624866
loss is :  1.3623571
loss is :  1.362228
loss is :  1.3620996
loss is :  1.3619719
loss is :  1.3618444
loss is :  1.3617177
loss is :  1.3615915
loss is :  1.3614657
loss is :  1.3613404
loss is :  1.3612156
loss is :  1.3610915
loss is :  1.3609678
loss is :  1.360844

loss is :  1.3363818
loss is :  1.3363483
loss is :  1.3363146
loss is :  1.3362812
loss is :  1.3362478
loss is :  1.3362144
loss is :  1.3361814
loss is :  1.3361483
loss is :  1.3361152
loss is :  1.3360825
loss is :  1.3360497
loss is :  1.336017
loss is :  1.3359845
loss is :  1.335952
loss is :  1.3359196
loss is :  1.3358874
loss is :  1.3358552
loss is :  1.3358234
loss is :  1.3357915
loss is :  1.3357595
loss is :  1.3357278
loss is :  1.3356962
loss is :  1.3356645
loss is :  1.3356332
loss is :  1.3356018
loss is :  1.3355705
loss is :  1.3355393
loss is :  1.3355083
loss is :  1.3354772
loss is :  1.3354464
loss is :  1.3354155
loss is :  1.335385
loss is :  1.3353544
loss is :  1.3353239
loss is :  1.3352937
loss is :  1.3352633
loss is :  1.335233
loss is :  1.3352027
loss is :  1.3351727
loss is :  1.335143
loss is :  1.335113
loss is :  1.3350832
loss is :  1.3350536
loss is :  1.335024
loss is :  1.3349944
loss is :  1.3349651
loss is :  1.3349358
loss is :  1.3349066

loss is :  1.3283087
loss is :  1.3282976
loss is :  1.3282868
loss is :  1.3282758
loss is :  1.328265
loss is :  1.3282541
loss is :  1.3282433
loss is :  1.3282326
loss is :  1.3282217
loss is :  1.328211
loss is :  1.3282003
loss is :  1.3281896
loss is :  1.328179
loss is :  1.3281683
loss is :  1.3281577
loss is :  1.328147
loss is :  1.3281366
loss is :  1.328126
loss is :  1.3281153
loss is :  1.3281049
loss is :  1.3280945
loss is :  1.328084
loss is :  1.3280735
loss is :  1.3280631
loss is :  1.3280528
loss is :  1.3280424
loss is :  1.3280321
loss is :  1.3280218
loss is :  1.3280115
loss is :  1.3280013
loss is :  1.327991
loss is :  1.3279808
loss is :  1.3279705
loss is :  1.3279603
loss is :  1.3279502
loss is :  1.3279402
loss is :  1.32793
loss is :  1.32792
loss is :  1.32791
loss is :  1.3278998
loss is :  1.3278898
loss is :  1.3278798
loss is :  1.3278699
loss is :  1.3278601
loss is :  1.3278501
loss is :  1.3278401
loss is :  1.3278303
loss is :  1.3278207
loss 

loss is :  1.3253179
loss is :  1.325313
loss is :  1.3253083
loss is :  1.3253034
loss is :  1.3252989
loss is :  1.325294
loss is :  1.3252894
loss is :  1.3252846
loss is :  1.32528
loss is :  1.3252753
loss is :  1.3252705
loss is :  1.3252658
loss is :  1.3252611
loss is :  1.3252565
loss is :  1.3252516
loss is :  1.325247
loss is :  1.3252424
loss is :  1.3252379
loss is :  1.3252332
loss is :  1.3252286
loss is :  1.3252239
loss is :  1.3252192
loss is :  1.3252147
loss is :  1.3252101
loss is :  1.3252056
loss is :  1.3252008
loss is :  1.3251963
loss is :  1.3251919
loss is :  1.3251872
loss is :  1.3251827
loss is :  1.3251781
loss is :  1.3251736
loss is :  1.3251691
loss is :  1.3251646
loss is :  1.32516
loss is :  1.3251555
loss is :  1.3251511
loss is :  1.3251464
loss is :  1.3251421
loss is :  1.3251375
loss is :  1.3251331
loss is :  1.3251287
loss is :  1.325124
loss is :  1.3251197
loss is :  1.3251152
loss is :  1.3251109
loss is :  1.3251065
loss is :  1.325102
l

...

loss is :  1.3209393
loss is :  1.3209392
loss is :  1.3209392
loss is :  1.3209391
loss is :  1.320939
loss is :  1.3209388
loss is :  1.3209388
loss is :  1.3209388
loss is :  1.3209387
loss is :  1.3209386
loss is :  1.3209386
loss is :  1.3209385
loss is :  1.3209383
loss is :  1.3209382
loss is :  1.3209382
loss is :  1.3209381
loss is :  1.320938
loss is :  1.3209379
loss is :  1.3209379
loss is :  1.3209378
loss is :  1.3209379
loss is :  1.3209376
loss is :  1.3209375
loss is :  1.3209374
loss is :  1.3209374
loss is :  1.3209372
loss is :  1.3209372
loss is :  1.3209372
loss is :  1.3209372
loss is :  1.320937
loss is :  1.320937
loss is :  1.3209368
loss is :  1.3209367
loss is :  1.3209367
loss is :  1.3209367
loss is :  1.3209367
loss is :  1.3209364
loss is :  1.3209364
loss is :  1.3209364
loss is :  1.3209361
loss is :  1.3209361
loss is :  1.3209361
loss is :  1.3209358
loss is :  1.3209358
loss is :  1.3209358
loss is :  1.3209358
loss is :  1.3209356
loss is :  1.3209

loss is :  1.3209085
loss is :  1.3209085
loss is :  1.3209084
loss is :  1.3209084
loss is :  1.3209083
loss is :  1.3209082
loss is :  1.3209082
loss is :  1.3209082
loss is :  1.3209081
loss is :  1.3209081
loss is :  1.3209078
loss is :  1.3209077
loss is :  1.3209077
loss is :  1.3209076
loss is :  1.3209076
loss is :  1.3209076
loss is :  1.3209076
loss is :  1.3209074
loss is :  1.3209074
loss is :  1.3209072
loss is :  1.320907
loss is :  1.320907
loss is :  1.320907
loss is :  1.3209069
loss is :  1.3209068
loss is :  1.3209068
loss is :  1.3209068
loss is :  1.3209066
loss is :  1.3209066
loss is :  1.3209065
loss is :  1.3209065
loss is :  1.3209065
loss is :  1.3209063
loss is :  1.3209062
loss is :  1.3209062
loss is :  1.3209062
loss is :  1.320906
loss is :  1.320906
loss is :  1.320906
loss is :  1.320906
loss is :  1.3209057
loss is :  1.3209057
loss is :  1.3209056
loss is :  1.3209054
loss is :  1.3209054
loss is :  1.3209054
loss is :  1.3209054
loss is :  1.3209053

loss is :  1.320881
loss is :  1.3208809
loss is :  1.3208809
loss is :  1.3208809
loss is :  1.3208809
loss is :  1.3208807
loss is :  1.3208807
loss is :  1.3208807
loss is :  1.3208805
loss is :  1.3208804
loss is :  1.3208804
loss is :  1.3208803
loss is :  1.3208803
loss is :  1.3208802
loss is :  1.3208802
loss is :  1.32088
loss is :  1.3208799
loss is :  1.3208799
loss is :  1.3208799
loss is :  1.3208799
loss is :  1.3208799
loss is :  1.3208798
loss is :  1.3208796
loss is :  1.3208796
loss is :  1.3208795
loss is :  1.3208793
loss is :  1.3208793
loss is :  1.3208793
loss is :  1.3208792
loss is :  1.3208793
loss is :  1.3208791
loss is :  1.3208791
loss is :  1.3208791
loss is :  1.3208789
loss is :  1.3208787
loss is :  1.3208787
loss is :  1.3208786
loss is :  1.3208786
loss is :  1.3208786
loss is :  1.3208786
loss is :  1.3208785
loss is :  1.3208785
loss is :  1.3208784
loss is :  1.3208781
loss is :  1.3208781
loss is :  1.3208781
loss is :  1.3208781
loss is :  1.320

loss is :  1.3208562
loss is :  1.3208563
loss is :  1.3208563
loss is :  1.3208561
loss is :  1.3208561
loss is :  1.320856
loss is :  1.3208559
loss is :  1.320856
loss is :  1.3208559
loss is :  1.3208559
loss is :  1.3208556
loss is :  1.3208556
loss is :  1.3208555
loss is :  1.3208555
loss is :  1.3208555
loss is :  1.3208555
loss is :  1.3208553
loss is :  1.3208553
loss is :  1.3208553
loss is :  1.3208553
loss is :  1.3208553
loss is :  1.3208551
loss is :  1.320855
loss is :  1.3208549
loss is :  1.320855
loss is :  1.3208549
loss is :  1.3208548
loss is :  1.3208548
loss is :  1.3208548
loss is :  1.3208547
loss is :  1.3208548
loss is :  1.3208545
loss is :  1.3208545
loss is :  1.3208545
loss is :  1.3208544
loss is :  1.3208542
loss is :  1.3208543
loss is :  1.3208542
loss is :  1.3208542
loss is :  1.3208541
loss is :  1.320854
loss is :  1.320854
loss is :  1.320854
loss is :  1.320854
loss is :  1.3208538
loss is :  1.3208537
loss is :  1.3208537
loss is :  1.3208536


loss is :  1.3208342
loss is :  1.3208342
loss is :  1.3208342
loss is :  1.3208342
loss is :  1.3208342
loss is :  1.320834
loss is :  1.3208339
loss is :  1.3208339
loss is :  1.3208338
loss is :  1.3208338
loss is :  1.3208338
loss is :  1.3208337
loss is :  1.3208337
loss is :  1.3208337
loss is :  1.3208336
loss is :  1.3208334
loss is :  1.3208334
loss is :  1.3208334
loss is :  1.3208334
loss is :  1.3208332
loss is :  1.3208331
loss is :  1.3208332
loss is :  1.3208331
loss is :  1.3208331
loss is :  1.320833
loss is :  1.320833
loss is :  1.3208328
loss is :  1.3208328
loss is :  1.3208327
loss is :  1.3208328
loss is :  1.3208327
loss is :  1.3208327
loss is :  1.3208325
loss is :  1.3208325
loss is :  1.3208325
loss is :  1.3208324
loss is :  1.3208324
loss is :  1.3208323
loss is :  1.3208323
loss is :  1.3208321
loss is :  1.3208323
loss is :  1.3208323
loss is :  1.3208321
loss is :  1.3208321
loss is :  1.3208321
loss is :  1.3208321
loss is :  1.3208318
loss is :  1.320

In [13]:
print(sess.run(W1))
print('----------')
print(sess.run(b1))
print('----------')


[[-0.03106876 -0.5626789   0.15172458 -0.92349905 -1.448336  ]
 [-0.18963596 -1.9572911  -2.0274227   0.40906426 -2.0153491 ]
 [-1.271095    0.96792465 -1.8345988   2.6283658  -1.7918696 ]
 [-1.1807425   0.17817862  0.4167314   1.1029863  -0.785998  ]
 [-0.50899005 -0.31409466  1.3500724   1.5497907  -0.64918363]
 [-0.81725484  0.31065404  0.11650766 -0.45862067 -0.04406165]
 [ 0.7809519  -0.5439079  -2.6756945   0.53238463  2.4164689 ]]
----------
[-0.6909525  1.3019264  1.7230113 -0.9102686 -0.813588 ]
----------


In [14]:
vectors = sess.run(W1 + b1)



In [15]:
def euclidean_dist(vec1, vec2):
    return np.sqrt(np.sum((vec1-vec2)**2))

def find_closest(word_index, vectors):
    min_dist = 10000 # to act like positive infinity
    min_index = -1
    query_vector = vectors[word_index]
    for index, vector in enumerate(vectors):
        if euclidean_dist(vector, query_vector) < min_dist and not np.array_equal(vector, query_vector):
            min_dist = euclidean_dist(vector, query_vector)
            min_index = index
    return min_index




In [16]:
print(int2word[find_closest(word2int['king'], vectors)])
print(int2word[find_closest(word2int['queen'], vectors)])
print(int2word[find_closest(word2int['royal'], vectors)])


she
king
she


In [ ]:
from sklearn.manifold import TSNE

model = TSNE(n_components=2, random_state=0)
np.set_printoptions(suppress=True)
vectors = model.fit_transform(vectors) 

from sklearn import preprocessing

normalizer = preprocessing.Normalizer()
vectors =  normalizer.fit_transform(vectors, 'l2')

print(vectors)

import matplotlib.pyplot as plt


fig, ax = plt.subplots()
print(words)
for word in words:
    print(word, vectors[word2int[word]][1])
    ax.annotate(word, (vectors[word2int[word]][0],vectors[word2int[word]][1] ))
plt.show()